In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from extra_data import RunDirectory
from extra_data.read_machinery import find_proposal
from extra_redu import make_litframe_finder, FrameSelection, SelType

In [ ]:
propno = 3118
runno = 18

propdir = find_proposal(f"p{propno:06d}")
rundir = os.path.join(propdir, f"raw/r{runno:04d}")

run = RunDirectory(rundir)
inst = rundir.split('/')[4]

In [ ]:
dev = make_litframe_finder(inst, run)

tm0 = time.monotonic()
r = dev.read_or_process()
tid = r.meta.trainId
tm1 = time.monotonic()

print(f"N trains: {tid.size}, time: {tm1-tm0:.2f}s, train per sec: {tid.size/(tm1-tm0):.2f}")

In [ ]:
sel = FrameSelection(r, guess_missed=True)

print(f"N train in selection: {len(sel.trains)}, N train in litframefinder: {tid.size}")
print(f"N selected frames: {sel.cell_flags.sum()}/{sel.cell_flags.size}, "
      f"ratio: {sel.cell_flags.size/sel.cell_flags.sum()}")

In [ ]:
a, b = np.unique(sel.cell_flags.reshape(-1, 352), axis=0, return_inverse=True)
nuniq = len(a)
c = b[:nuniq][None,:]

nn = (b.size // nuniq) * nuniq
thesame = np.all(b[:nn].reshape(-1,nuniq) == c)

print(f"N unique patterns: {nuniq}, first {nuniq} indices: {c}")
print(f"Paterns repeats: {thesame}")

In [ ]:
def make_cell_ids(count):
    cid = np.zeros(np.sum(count), np.uint16)
    i0 = 0
    for nfrm in count:
        iN = i0 + nfrm
        cid[i0:iN] = np.arange(nfrm, dtype=np.uint16)
        i0 = iN
    return cid

In [ ]:
ntrn = 100
slc = np.s_[:ntrn]
cid = make_cell_ids(sel.count[slc])
flags = sel.litframes_on_trains(tid[slc], sel.count[slc], cid, [SelType.CELL, SelType.ROW])
for fl, cnt in flags:
    plt.matshow(fl.reshape(ntrn, -1))
    plt.show()

In [ ]:
ntrn = 100
slc = np.s_[:ntrn*2:2]
cid = make_cell_ids(sel.count[slc])
flags = sel.litframes_on_trains(tid[slc], sel.count[slc], cid,
                                 [SelType.CELL, SelType.SUPER_CELL, SelType.SUPER_ROW])
for fl, cnt in flags:
    plt.matshow(fl.reshape(ntrn, -1))
    plt.show()